In [1]:
import pandas as pd

df = pd.read_csv('pollution_data.csv')
df['created_at'] = pd.to_datetime(df['created_at'])
df['date'] = df['created_at']
df.set_index('date', inplace=True)

display(df.head(10))

,created_at,Temperature,Humidity,PM2.5,PM10,CO,CO2
date,,,,,,,
2025-06-21 07:00:15+07:00,2025-06-21 07:00:15+07:00,24.8,84.3,52.7,72.5,0.59860,403.34171
2025-06-21 07:01:04+07:00,2025-06-21 07:01:04+07:00,24.8,84.1,50.4,66.7,0.56109,403.40472
2025-06-21 07:01:53+07:00,2025-06-21 07:01:53+07:00,24.8,84.0,51.7,74.0,0.54466,403.36261
2025-06-21 07:02:43+07:00,2025-06-21 07:02:43+07:00,24.9,83.6,49.7,65.9,0.55447,403.30020
2025-06-21 07:03:32+07:00,2025-06-21 07:03:32+07:00,24.9,83.3,51.6,72.7,0.57113,403.29333
2025-06-21 07:04:22+07:00,2025-06-21 07:04:22+07:00,24.9,83.3,48.3,67.9,0.56609,403.29333
2025-06-21 07:05:11+07:00,2025-06-21 07:05:11+07:00,24.9,83.0,48.7,65.8,0.55612,403.29333
2025-06-21 07:06:03+07:00,2025-06-21 07:06:03+07:00,24.9,83.0,50.2,67.4,0.55119,403.32782
2025-06-21 07:06:53+07:00,2025-06-21 07:06:53+07:00,24.9,82.9,52.0,69.8,0.53338,403.21170


In [2]:
df = df.drop(columns=['Temperature', 'Humidity', 'created_at'])
df_hourly = df.resample('3T').mean()
display(df_hourly.head(10))

/tmp/ipython-input-2-1558996870.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_hourly = df.resample('3T').mean()


,PM2.5,PM10,CO,CO2
date,,,,
2025-06-21 07:00:00+07:00,51.125000,69.775000,0.564705,403.352310
2025-06-21 07:03:00+07:00,49.533333,68.800000,0.564447,403.293330
2025-06-21 07:06:00+07:00,50.475000,68.875000,0.545560,403.244147
2025-06-21 07:09:00+07:00,55.525000,74.900000,0.553152,403.269515
2025-06-21 07:12:00+07:00,53.966667,70.733333,0.548513,403.360290
2025-06-21 07:15:00+07:00,53.950000,71.600000,0.567908,403.354330
2025-06-21 07:18:00+07:00,55.650000,72.775000,0.553165,403.292860
2025-06-21 07:21:00+07:00,58.700000,78.000000,0.555143,403.342750
2025-06-21 07:24:00+07:00,62.225000,79.525000,0.564465,403.410062


In [3]:
from sklearn.preprocessing import MinMaxScaler

features = ['PM2.5', 'PM10', 'CO', 'CO2']
data = df_hourly[features].dropna()

scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

In [4]:
import numpy as np

def create_sequences(data, seq_len=20):
    X, y = [], []
    for i in range(len(data) - seq_len):
        X.append(data[i:(i + seq_len)])
        y.append(data[i + seq_len])
    return np.array(X), np.array(y)

seq_len = 20
X, y = create_sequences(scaled_data, seq_len)

X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

model = Sequential()
model.add(LSTM(64, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(4))  # 4 output: PM2.5, PM10, CO, CO2
model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError())

model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


263/263 ━━━━━━━━━━━━━━━━━━━━ 7s 18ms/step - loss: 0.0117 - val_loss: 0.0016
Epoch 2/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 3/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 8.5820e-04 - val_loss: 0.0010
Epoch 4/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - loss: 7.1330e-04 - val_loss: 8.1211e-04
Epoch 5/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 10ms/step - loss: 6.7172e-04 - val_loss: 9.5425e-04
Epoch 6/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - loss: 5.6885e-04 - val_loss: 7.9781e-04
Epoch 7/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - loss: 5.5766e-04 - val_loss: 7.9243e-04
Epoch 8/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 6.2285e-04 - val_loss: 6.5719e-04
Epoch 9/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - loss: 5.0095e-04 - val_loss: 6.5299e-04
Epoch 10/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - loss: 4.8408e-04 - val_loss: 6.1202e-04
Epoch 11/100
263/263 ━━━━━━━━━━━━━━━━━━━━ 7s 26ms/step - loss: 4.7928e

In [11]:
# Prediksi semua data (untuk evaluasi)
y_pred_scaled = model.predict(X)

# Kembalikan ke skala asli
y_pred = scaler.inverse_transform(y_pred_scaled)
y_true = scaler.inverse_transform(y)

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_true, y_pred)
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
r2 = r2_score(y_true, y_pred)

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")


329/329 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
MAE: 2.20
RMSE: 6.50
R²: 0.96


In [12]:
# Simpan model
model.save("lstm_model.h5")

# Simpan scaler
import joblib
joblib.dump(scaler, "scaler.save")


['scaler.save']

In [13]:
from tensorflow.keras.models import load_model
import joblib
import numpy as np

# Load model dan scaler
model = load_model("lstm_model.h5")
scaler = joblib.load("scaler.save")

recent_data = np.array([
    [29.1, 30.2, 1.1, 462],
    [30.5, 32.3, 0.7, 438],
    [31.8, 33.7, 1.8, 492],
    [29.6, 30.9, 0.5, 422],
    [32.4, 34.1, 2.0, 508],
    [30.2, 31.5, 0.9, 448],
    [33.3, 35.0, 1.5, 478],
    [30.8, 32.7, 1.0, 454],
    [31.6, 33.3, 1.3, 470],
    [34.0, 36.2, 2.2, 510],
    [29.7, 30.8, 0.6, 428],
    [32.1, 34.0, 1.9, 500],
    [30.0, 31.1, 0.8, 442],
    [33.5, 35.3, 1.6, 484],
    [31.9, 32.8, 1.2, 466],
    [28.6, 30.0, 0.4, 418],
    [34.3, 36.0, 2.1, 504],
    [30.9, 31.7, 1.1, 460],
    [31.2, 32.5, 1.2, 468],
    [29.9, 31.0, 0.7, 440]
])  # Bentuk: (20 (3 menit x 1 jam), 4 fitur)

recent_scaled = scaler.transform(recent_data)
recent_scaled = recent_scaled.reshape((1, 20, 4))  # (1 sample, 20 time steps, 4 features)

pred_scaled = model.predict(recent_scaled)
predicted = scaler.inverse_transform(pred_scaled)  # Kembali ke skala asli

print("Prediksi kualitas udara 1 jam ke depan:")
print(f"PM2.5: {predicted[0][0]:.2f}")
print(f"PM10 : {predicted[0][1]:.2f}")
print(f"CO   : {predicted[0][2]:.2f}")
print(f"CO2  : {predicted[0][3]:.2f}")


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step


array([[0.28107533, 0.2863102 , 0.94826275, 2.1944714 ]], dtype=float32)

array([[ 63.170643 ,  73.07022  ,   2.7696204, 423.6568   ]],
      dtype=float32)

Prediksi kualitas udara 1 jam ke depan:
PM2.5: 63.17
PM10 : 73.07
CO   : 2.77
CO2  : 423.66


In [ ]:
features = ['PM2.5', 'PM10', 'CO', 'CO2']
look_back = 20
n_features = len(features)

# Ambil 20 langkah terakhir
recent_data = df_hourly[features].dropna().tail(look_back).values

def predict_20x3min(model, scaler, initial_seq, look_back=20):
    preds = []
    seq = initial_seq.copy()
    for _ in range(20):  # 20 langkah ke depan
        scaled = scaler.transform(seq)
        input_seq = scaled.reshape((1, 20, n_features))
        pred_scaled = model.predict(input_seq)
        pred = scaler.inverse_transform(pred_scaled)[0]
        preds.append(pred)
        seq = np.vstack([seq[1:], pred])
    return np.array(preds)

# Jalankan prediksi
predictions = predict_20x3min(model, scaler, recent_data)

# Tampilkan hasil prediksi
for i, step in enumerate(predictions):
    print(f"Prediksi T+{(i)*3} menit:")
    print(f"  PM2.5: {step[0]:.2f}")
    print(f"  PM10 : {step[1]:.2f}")
    print(f"  CO   : {step[2]:.2f}")
    print(f"  CO2  : {step[3]:.2f}")
    print("-" * 30)
